In [ ]:
import requests
import urllib.request
import json
import pandas as pd
import time
from datetime import datetime, timedelta
from sqlalchemy import create_engine, event
from urllib.parse import quote_plus
import sys
import datetime as DT
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import re

In [ ]:
scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file('lively-clover-265718-6f7adafc0648.json', scopes=scopes)

gc = gspread.authorize(credentials)

gauth = GoogleAuth()
drive = GoogleDrive(gauth)

In [ ]:
API_KEY = ""
CLIENT_ID = '515935'
hdrs = {"Client-Id": CLIENT_ID, "Api-Key": API_KEY,"Content-Type": "application/json"}

In [ ]:
#Выгружаем данные из FINANCE_REPORT которые уже есть у нас

In [ ]:
# open a google sheet
gs = gc.open_by_key('')

# select a work sheet from its name
worksheet1 = gs.worksheet('FINANCE_REPORT')

#Преобразуем в Dataframe
finance_report = pd.DataFrame(worksheet1.get_all_records())

#берем нужные колонки

finance_report = finance_report.loc[:,:'posting warehouse_id']

In [ ]:
#Печатаем нужные даты 

print(finance_report['operation_date'].max(),datetime.today().strftime('%Y-%m-%d'))

In [ ]:
#Выгружаем отчет по транзакциям за нужный период

url = "https://api-seller.ozon.ru/v3/finance/transaction/list"

date_from = "2023-05-30T00:00:00.000Z" #дата на день больше чем в нашем отчете
date_to = "2023-05-31T00:00:00.000Z" #дата на день меньше чем сегодняшняя

body = {
        "filter": {
            "date": {
                "from": date_from,
                "to": date_to
            },
            "operation_type": [],
            "posting_number": "",
            "transaction_type": "all"
        },
        "page": 1,
        "page_size": 1000
    }
body = json.dumps(body)

report_data = requests.post(url, headers=hdrs, data=body)
json_report_data = report_data.json()

In [ ]:
report_data.text

In [ ]:

data_to_df = json_report_data["result"]["operations"]
df = pd.json_normalize(data_to_df, errors='ignore', sep='.', max_level=None)
#df = df.drop(columns='items')
df = df.drop(columns='services')

df['items'] = df['items'].apply(lambda x: re.sub("[',[,{,:]", " ", str(x)).replace('name','').replace(']','').replace('sku',''))

In [ ]:
df

In [ ]:
#Добавляем в гугл шит

df_values = df.values.tolist()
gs.values_append('FINANCE_REPORT', {'valueInputOption': 'RAW'}, {'values': df_values})

In [ ]:
#Берем имеющиеся у нас данные по Orders

# open a google sheet
gs = gc.open_by_key('')

# select a work sheet from its name
worksheet1 = gs.worksheet('ORDERS')

#Преобразуем в Dataframe
orders_report = pd.DataFrame(worksheet1.get_all_records())


In [ ]:
orders_report

In [ ]:
print(orders_report['Дата'].max(),datetime.today().strftime('%Y-%m-%d'))

In [ ]:
#Удаляем последний месяц

orders_report = orders_report.drop(orders_report[orders_report['Дата'] >='2023-05-01'].index)


In [ ]:
#Записываем в гугл шит

#Записываем данные в гугл шит
# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('ORDERS')

# write to dataframe

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=orders_report, include_index=False,include_column_header=True, resize=True)

In [ ]:
#Выгружаем данные из Orders за месяц до вчерашнего числа

In [ ]:
url = "https://api-seller.ozon.ru/v1/report/postings/create"
url_2 = "https://api-seller.ozon.ru/v1/report/info"
date_from = "2023-05-01T00:00:00.000Z"
date_to = "2023-05-31T00:00:00.000Z"

body = {
        "filter": {
    "processed_at_from": date_from,
    "processed_at_to": date_to,
    "delivery_schema": [
    "fbs",
    "fbo",
    "crossborder"
],
    "sku": [ ],
    "cancel_reason_id": [ ],
    "offer_id": "",
    "status_alias": [ ],
    "statuses": [ ],
    "title": ""
},
"language": "DEFAULT"
    }

body = json.dumps(body)
report_data = requests.post(url, headers=hdrs, data=body)
json_report_data = report_data.json()

In [ ]:
json_report_data

In [ ]:
json_report_data['result']['code']

In [ ]:
report_data = requests.post(url_2, headers=hdrs, json=json_report_data['result'])
json_report_data = report_data.json()

In [ ]:
report_data.text

In [ ]:
urllib.request.urlretrieve(json_report_data['result']['file'], "Orders.csv")

In [ ]:
orders_df = pd.read_csv("Orders.csv",sep=';')

In [ ]:
orders_df

In [ ]:
#Добавляем в гугл шит

orders_df_list = orders_df.values.tolist()
gs.values_append('ORDERS', {'valueInputOption': 'RAW'}, {'values': orders_df_list})

In [ ]:
#Берем максимальную дату из TOTAL_FINANCE


# open a google sheet
gs = gc.open_by_key('')

# select a work sheet from its name
worksheet1 = gs.worksheet('TOTAL_FINANCE')

#Преобразуем в Dataframe
TOTAL_FINANCE = pd.DataFrame(worksheet1.get_all_records())


print(TOTAL_FINANCE['operation_date'].max())

In [ ]:
orders_df

In [ ]:
orders_df = orders_df[['Номер отправления','Статус','Артикул','Цена','Количество']]
final_finance = df.merge(orders_df,how='left',left_on='posting.posting_number', right_on='Номер отправления')

In [ ]:
#Добавляем в гугл шит
final_finance = final_finance.fillna('')

final_finance_list = final_finance.values.tolist()
gs.values_append('TOTAL_FINANCE', {'valueInputOption': 'RAW'}, {'values': final_finance_list})

In [ ]:
#Объединяем нашу выгрузку с выгрузкой из озона и убираем дубли

orders_final = pd.concat([orders_report,orders_df]).drop_duplicates()

In [ ]:
orders_final[orders_final.duplicated()]

In [ ]:
orders_final

In [ ]:
#Записываем данные в гугл шит
# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('ORDERS')

# write to dataframe

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=orders_final, include_index=False,include_column_header=True, resize=True)

In [ ]:
#Берем нужные колонки из orders
total_orders_1 = orders_final[['Номер отправления','Статус','Артикул','Цена','Количество']]

#Объединяем их с FINANCE
final_finance = df.merge(total_orders_1,how='left',left_on='posting.posting_number', right_on='Номер отправления')

In [ ]:
final_finance = final_finance.fillna('')

In [ ]:
final_finance.drop_duplicates(inplace=True)

In [ ]:
final_finance[final_finance.duplicated()]

In [ ]:
#Записываем данные в гугл шит
# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('FINANCE_REPORT')
#Дозаписываем в гугл шит

df_values = final_finance.values.tolist()
gs.values_append('FINANCE_REPORT', {'valueInputOption': 'RAW'}, {'values': df_values})


In [ ]:
df_values

In [ ]:
total_orders = pd.concat([orders_df,orders_df_2,orders_df_3])

In [ ]:
total_orders.drop_duplicates()

In [ ]:
# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('ORDERS')

In [ ]:
# write to dataframe

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=total_orders, include_index=False,include_column_header=True, resize=True)

In [ ]:
total_orders_1 = total_orders[['Номер отправления','Статус','Артикул','Цена','Количество']]

In [ ]:
final_finance = total_df.merge(total_orders_1,how='left',left_on='posting.posting_number', right_on='Номер отправления')

In [ ]:
final_finance

In [ ]:
# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('FINANCE_REPORT')

In [ ]:
# write to dataframe

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=final_finance, include_index=False,include_column_header=True, resize=True)

In [ ]:
#Запись в гугл таблицу

In [ ]:
scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file('lively-clover-265718-6f7adafc0648.json', scopes=scopes)

gc = gspread.authorize(credentials)

gauth = GoogleAuth()
drive = GoogleDrive(gauth)

# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('FINANCE_REPORT')


In [ ]:
# write to dataframe

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=total_df, include_index=False,include_column_header=True, resize=True)

In [ ]:
#Append values

df_values = total_df.values.tolist()
gs.values_append('FINANCE_REPORT', {'valueInputOption': 'RAW'}, {'values': df_values})

In [ ]:
Orders 
#https://docs.google.com/spreadsheets/d/'1_24Q5hstJ7Ib15XMXOgcDAgqRgXoc83mnm9uRm96N9s'/edit?usp=sharing

In [ ]:
url = "https://api-seller.ozon.ru/v1/report/postings/create"
url_2 = "https://api-seller.ozon.ru/v1/report/info"
date_from = "2023-03-01T00:00:00.000Z"
date_to = "2023-04-30T00:00:00.000Z"

body = {
        "filter": {
    "processed_at_from": date_from,
    "processed_at_to": date_to,
    "delivery_schema": [
    "fbs",
    "fbo",
    "crossborder"
],
    "sku": [ ],
    "cancel_reason_id": [ ],
    "offer_id": "",
    "status_alias": [ ],
    "statuses": [ ],
    "title": ""
},
"language": "DEFAULT"
    }

body = json.dumps(body)
report_data = requests.post(url, headers=hdrs, data=body)
json_report_data = report_data.json()

In [ ]:
json_report_data

In [ ]:
json_report_data['result']['code']

In [ ]:
report_data = requests.post(url_2, headers=hdrs, json=json_report_data['result'])
json_report_data = report_data.json()

In [ ]:
report_data.text

In [ ]:
urllib.request.urlretrieve(json_report_data['result']['file'], "Orders.csv")

In [ ]:
orders_df_2 = pd.read_csv("Orders.csv",sep=';')

In [ ]:
orders_df_2

In [ ]:
total_orders = pd.concat([orders_df,orders_df_2])

In [ ]:
total_orders.drop_duplicates()

In [ ]:
# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('ORDERS')

In [ ]:
# write to dataframe

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=total_orders, include_index=False,include_column_header=True, resize=True)

In [ ]:
total_orders = total_orders[['Номер отправления','Статус','Артикул','Цена','Количество']]

In [ ]:
total_orders

In [ ]:
final_finance = df_apr.merge(total_orders,how='left',left_on='posting.posting_number', right_on='Номер отправления')

In [ ]:
final_finance

In [ ]:

# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('FINANCE_REPORT')


In [ ]:

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=final_finance, include_index=False,include_column_header=True, resize=True)

In [ ]:
#Остатки FBO 2

In [ ]:
url = "https://api-seller.ozon.ru/v2/analytics/stock_on_warehouses"
url_2 = "https://api-seller.ozon.ru/v1/report/info"
#date_from = "2023-02-01T00:00:00.000Z"
#date_to = "2023-04-27T00:00:00.000Z"

body = {
        
"limit": 1000,
"offset": 0,
"warehouse_type": "ALL"
    }

body = json.dumps(body)
report_data = requests.post(url, headers=hdrs, data=body)
json_report_data = report_data.json()

In [ ]:
json_report_data

In [ ]:
data_to_df = json_report_data["result"]["rows"]
df = pd.json_normalize(data_to_df, errors='ignore', sep='.', max_level=None)
#df = df.drop(columns='items')
#df = df.drop(columns='services')

In [ ]:
# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('FBO STOCK')


In [ ]:

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=df, include_index=False,include_column_header=True, resize=True)

In [ ]:
#Остатки на складах FBO

In [ ]:
url = "https://api-seller.ozon.ru/v1/report/stock/create"
url_2 = "https://api-seller.ozon.ru/v1/report/info"
#date_from = "2023-02-01T00:00:00.000Z"
#date_to = "2023-04-27T00:00:00.000Z"

body = {
        
"language": "DEFAULT"
    }

body = json.dumps(body)
report_data = requests.post(url, headers=hdrs, data=body)
json_report_data = report_data.json()

In [ ]:
json_report_data

In [ ]:
json_report_data['result']['code']

In [ ]:
report_data = requests.post(url_2, headers=hdrs, json=json_report_data['result'])
json_report_data = report_data.json()

In [ ]:
report_data.text

In [ ]:
urllib.request.urlretrieve(json_report_data['result']['file'], "Stock.csv")

In [ ]:
Stock_df = pd.read_csv("Stock.csv",sep=';')

In [ ]:
Stock_df

In [ ]:
#Остатки FBO 2

In [ ]:
url = "https://api-seller.ozon.ru/v2/analytics/stock_on_warehouses"
url_2 = "https://api-seller.ozon.ru/v1/report/info"
#date_from = "2023-02-01T00:00:00.000Z"
#date_to = "2023-04-27T00:00:00.000Z"

body = {
        
"limit": 1000,
"offset": 0,
"warehouse_type": "ALL"
    }

body = json.dumps(body)
report_data = requests.post(url, headers=hdrs, data=body)
json_report_data = report_data.json()

In [ ]:
json_report_data

In [ ]:
data_to_df = json_report_data["result"]["rows"]
df = pd.json_normalize(data_to_df, errors='ignore', sep='.', max_level=None)
#df = df.drop(columns='items')
#df = df.drop(columns='services')

In [ ]:
df

In [ ]:
# open a google sheet
gs = gc.open_by_key('')
# select a work sheet from its name
worksheet1 = gs.worksheet('FBO STOCK')


In [ ]:

worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=df, include_index=False,include_column_header=True, resize=True)

In [ ]:
#Отправления по FBO

In [ ]:
url = "https://api-seller.ozon.ru/v2/posting/fbo/list"
url_2 = "https://api-seller.ozon.ru/v1/report/info"
date_from = "2023-04-01T00:00:00.000Z"
date_to = "2023-04-27T00:00:00.000Z"

body = {
        
"dir": "ASC",
"filter": {
"since": date_from,
"status": "",
"to": date_to
},
"limit": 1000,
"offset": 0,
"translit": True,
#"with": {
#"analytics_data": True,
#"financial_data": True
#}
    }

body = json.dumps(body)
report_data = requests.post(url, headers=hdrs, data=body)
json_report_data = report_data.json()

In [ ]:
json_report_data

In [ ]:
data_to_df = json_report_data["result"]
df = pd.json_normalize(data_to_df, errors='ignore', sep='.', max_level=None)

In [ ]:
df

In [ ]:
#Отправления по FBS

In [ ]:
https://github.com/lifeindarkside/OZON-reports-py/blob/main/ozon_main.py